In [150]:
import pandas as pd ; 
import numpy as np 
import time , os , sys , random
import sqlite3 , re , copy 
import matplotlib.pyplot as plt


def convert_db_data_to_csv(table_name , connection  , path = './data/' , filename="mydata.csv" ):
    df = pd.DataFrame(pd.read_sql('select * from '+table_name , connection)) ; 
    df.to_csv(os.path.join(path , filename)) ;
    

In [262]:
class allotment:
    def __init__(self , subjectid ,  facultyid1 , facultyid2 , roomid , dayno , hour , subjectname='' ):
        self.subjectid =subjectid ; 
        self.subjectname = subjectname
        self.facultyid1 = facultyid1
        self.facultyid2 = facultyid2
        self.roomid = roomid
        self.dayno = dayno
        self.hour = hour
        

class faculty:
    def __init__(self , fid , fname , ftype):
        self.id , self.name , self.type = fid , fname , ftype ; 
    def __str__(self):
        return "< ID : {} , Name : {}  , type : {}  >".format(self.id , self.name , self.type) ;
    def __repr__(self):
        return self.__str__() ;
        
        


class Timetable:
    
    def __init__(self):
        self.faculty_data = pd.read_csv("data/faculty.csv") ; 
        self.room_data = pd.read_csv('data/room.csv') ; 
        self.subject_data = pd.read_csv('data/subject.csv') ;
        self.test_data = pd.read_csv('data/test.csv') ;
        
        self.faculty_data = self.faculty_data[['facultyId' , 'facultyName' , 'facultyType']]
        self.room_data = self.room_data[['roomId' , 'roomType' , 'roomNo']]
        self.subject_data = self.subject_data[['subjectId' , 'subjectName' , 'subjectType' , 'credits']]
        self.test_data = self.test_data[['batchID' , 'subjectID' , 'facultyID1' , 'facultyID2']]
        
        
#         type : pd.DataFrame
        self.normal_rooms = self.room_data[(self.room_data.roomType==0) & (self.room_data.roomNo)]
        self.lab_rooms = self.room_data[(self.room_data.roomType==1) & (self.room_data.roomNo)]

        
#         lab_subjects : type = np.array
        self.lab_subjects = self.subject_data[self.subject_data.subjectType==1].subjectId.values
        self.normal_subjects = self.subject_data[self.subject_data.subjectType==0].subjectId.values
        
#         type:np.array
        self.sectionslist = obj.test_data[obj.test_data.subjectID.apply(lambda sid : True if sid in self.normal_subjects else False)].batchID.unique()
        
    
#       type: dict
        self.section_to_subjects = {}
        self.populate_section_to_subject() ; 
                
        
#         method calls 
        self.create_faculty_objects() ; 
        self.init_faculties() ;
        

        
        
        
#         returns facultyid if he is free else None
    def get_free_faculty(self , facultylist , day , hour ):
        '''Returns a random faculty from a np.array list of faculties who is free at that day and hour '''
        
        faculties = facultylist.values
        
        try:
            for faculty in faculties:
                if(self.faculty_to_day_hour_slot_map.get(faculty).get(day).get(hour).get('alloted') == False):
                    self.faculty_to_day_hour_slot_map.get(faculty).get(day).get(hour)['alloted'] = True
                    return faculty ;
        
        except AttributeError as err:
            print(err , 'faculty = ' , faculty , ' ,  day , hour = ' , day , hour ) ;  
        except Exception as Exp:
            print(Exp) ; 
            
        
        
    
#     populates the section_to_subjects dict  , maps section string to subjects list
    def populate_section_to_subject(self):
        for section in self.sectionslist:
            self.section_to_subjects[section] = self.test_data[self.test_data.batchID==section].subjectID ;
            
    
# #      return type: np.array 
#     def get_sections_from_batches(self):
#         '''Assume the string till last but one char as the section name and if there is only 1 char take that as section name'''
#         def myfun(x , compiledre):
#             if(len(x)==1):
#                 return x ;
#             else:
#                 if(compiledre.match(x)):
#                     return x;  
        
#         compiledre = re.compile('^.*?[a-zA-Z]$') ;
#         sections = self.test_data.batchID.apply(lambda x : myfun(x , compiledre)) ;
#         sectionslist = sections[sections.apply(lambda x : True if x else False)].unique()
#         return sectionslist ;
        

        
    def init_faculties(self):
        '''Initialize the faculties to be not allocated to all the hours of all days'''        
        self.faculty_to_day_hour_slot_map = {}
        day_to_hour = {}
        faculty_to_data = {} 

        hour_to_allotflag = {}
        for hour in range(1 , 9):
            hour_to_allotflag.update({hour : copy.deepcopy({'alloted': False})}) ;

        for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
            day_to_hour.update({day : copy.deepcopy(hour_to_allotflag)}) ;

        for faculty_object in self.faculties:
            self.faculty_to_day_hour_slot_map.update({faculty_object.id : copy.deepcopy(day_to_hour)}) ;
        
                    
#     return type : None
    def allot_slots_normal_class(self):
        
        allotment = pd.DataFrame(columns=['day' ,'section', 'hour', 'subjectid' , 'roomno' , 'facultyid']) ;
        
        for section in self.sectionslist:
            selected_room = self.normal_rooms.roomNo.sample().values[0] ;
            section_subjects_original = self.section_to_subjects[section].values ;
            
            for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
                batch_subs = section_subjects_original ; 

                for hour in range(1 , 9):
                    if(not batch_subs.any()):
                        selected_subject = self.section_to_subjects[section].sample().values[0]
                    else:
                        selected_subject_index =  random.randrange(len(batch_subs));
                        selected_subject = batch_subs[selected_subject_index] ; 
                        batch_subs = np.delete(batch_subs , selected_subject_index) ;
                        
                    
                    selected_faculty = self.get_free_faculty(self.test_data[self.test_data['subjectID']==selected_subject].facultyID1 , day= day , hour = hour) 
#                     self.faculty_to_day_hour_slot_map.get(selected_faculty).get(hour).update({'day' : day , 'hour' : hour , 'section' : section , 'subject' : subject})

#                     selected_faculty = self.test_data[self.test_data['subjectID']==selected_subject].facultyID1.sample().values[0]
                        
                    allotment = allotment.append({'section' : section , 'day' : day , 'hour' : hour  , 'subjectid' : selected_subject , 'roomno' : selected_room , 'facultyid' : selected_faculty  } , ignore_index=True)
                    
        self.non_lab_allotment = allotment ; 
                    
    
    
    def create_faculty_objects(self):
        '''Create facultie objects using the data ''' 
        self.faculties = set() ;
        for i in range(self.faculty_data.count().facultyId):
            self.faculties.add(faculty(self.faculty_data.loc[i].facultyId , self.faculty_data.loc[i].facultyName , self.faculty_data.loc[i].facultyType))
                
        
                    
    
    def check_allotment_validity(self , allotment_dataframe = None ):
        '''checks if the allotment DataFrame satisfies the hard constraints '''
        if(not allotment_dataframe):
            allotment_dataframe = self.non_lab_allotment ; 
        
        df = allotment_dataframe ;
        print("Starting faculty duplicate check for each hour of some day ") ; 
        
        for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
            for hour in range(1 , 9):
                temp = df[(df.hour==1) & (df.day==day)]
                temp = temp[temp.facultyid.apply(lambda x : True if x else False )].facultyid
                if(not (temp.nunique()==temp.count())):
                    print("Fails faculty non duplicate test : day , hour = " , day , hour) ; 
                    print("temp = " , temp) ;
                    print("returning .. ") ; 
                    return  False ;
        
        print("\nSUCCESS ^_^ ") ; 
            

obj = Timetable()

In [263]:
obj.allot_slots_normal_class()

In [264]:
obj.check_allotment_validity()

Starting faculty duplicate check for each hour of some day 

SUCCESS ^_^ 


In [266]:
obj.non_lab_allotment

,day,section,hour,subjectid,roomno,facultyid
0,mon,4A,1,4CCI02,CSL105,5
1,mon,4A,2,4CCI04,CSL105,47
2,mon,4A,3,FR,CSL105,42
3,mon,4A,4,4MAT2,CSL105,46
4,mon,4A,5,MC03,CSL105,48
5,mon,4A,6,4CCI03,CSL105,21
6,mon,4A,7,4CS01,CSL105,39
7,mon,4A,8,4CCI01,CSL105,29
8,tue,4A,1,4CCI04,CSL105,47
9,tue,4A,2,4CCI01,CSL105,29


In [116]:
normal_subs = obj.subject_data[obj.subject_data.subjectType==0].subjectId.values
normal_subs

array(['3CCI01', '3CCI02', '3CCI03', '3CCI04', '3MAT3C', '4CCI01',
       '4CCI02', '4CCI03', '4CCI04', '4CS01', '4MAT2', '5CCI01 ',
       '5CCI02', '5CCI03', '5CCI04', '6CCI01 ', '6CCI02', '6CS01',
       '6CS03', '7CCI01', '7CS02', 'ASE', 'AWN', 'CSPE01', 'CSPE02',
       'CSPE03', 'CSPE05', 'CSPE06', 'CSPE07', 'CSPE08', 'CSPE09',
       'CSPE10', 'CSPE11', 'CSPE12', 'CSPE13', 'CSPE14', 'CSPE15',
       'CSPE16', 'CSPE17', 'CSPE19', 'CSPE20', 'CSPE21', 'CSPE22',
       'CSPE23', 'CSPE24', 'CSPE25', 'CSPE26', 'CSPE27', 'CSPE28',
       'CSPE29', 'CSPE30', 'CSPE31', 'CSPE32', 'CSPE33', 'DCS', 'FR',
       'MC03', nan, 'OE01', 'OE02', 'OE03', 'ON', 'PE01', 'PE02', 'PE03',
       'PE04', 'PE05', 'PE06', 'SC'], dtype=object)

In [253]:
obj.test_data[obj.test_data.subjectID.apply(lambda sid : True if sid in normal_subs else False)].batchID.unique()

array(['4A', '4B', '4C', '6A', '6B', '6C', '8', 'M-CNE', 'M-CSE'],
      dtype=object)

9